Objective: get_history should fetch all the data at once then save it to separate files.

In [ ]:
from audiolizer import get_history

In [ ]:
help(get_history)

In [ ]:
from audiolizer import HistoricalData
from audiolizer import granularity, audiolizer_temp_dir, get_gaps, load_date
import pandas as pd
import os


def fetch_data(ticker, granularity, start_, end_):
    """Need dates in this format %Y-%m-%d-%H-%M"""
    try:
        return HistoricalData(ticker,
                              granularity,
                              start_,
                              end_,
                              ).retrieve_data()
    except:
        print('could not load using {} {}'.format(start_, end_))
        raise


def write_data(df):
    for t, day in df.groupby(pd.Grouper(freq='1D')):
        tstr = t.strftime('%Y-%m-%d-%H-%M')
        fname = audiolizer_temp_dir + '/{}-{}.csv.gz'.format(
                ticker, t.strftime('%Y-%m-%d'))
        day.to_csv(fname, compression='gzip')
        print('wrote {}'.format(fname))
        
def fetch_missing(files_status):
    """Iterate over batches of missing dates"""
    for batch, g in files_status[files_status.found==0].groupby('batch', sort=False):
        t1, t2 = g.iloc[[0,-1]].index
        if t1 == t2:
            t2 = t1 + pd.Timedelta('1D')
        endpoints = [t.strftime('%Y-%m-%d-%H-%M') for t in [t1, t2]]
        print('fetching {}, {}'.format(len(g), endpoints))
        df = fetch_data(ticker, granularity, *endpoints)
        write_data(df)

def get_history(ticker, start_date, end_date = None, granularity=granularity):
    """Fetch/load historical data from Coinbase API at specified granularity

    params:
        start_date: (str) (see pandas.to_datetime for acceptable formats)
        end_date: (str)
        granularity: (int) seconds (default: 300)

    price data is saved by ticker and date and stored in audiolizer_temp_dir
    """
    start_date = pd.to_datetime(start_date).tz_localize(None)
    
    today = pd.Timestamp.now().tz_localize(None)
    if end_date is None:
        end_date = today + pd.Timedelta('1D')
    else:
        end_date = min(today, pd.to_datetime(end_date).tz_localize(None))
        
    fnames = []
    foundlings = []
    dates = []
    batch = []
    batch_number = 0
    last_found = -1
    for int_ in pd.interval_range(start_date, end_date):
        dates.append(int_.left)
        fname = audiolizer_temp_dir + '/{}-{}.csv.gz'.format(
            ticker, int_.left.strftime('%Y-%m-%d'))
        found = int(os.path.exists(fname))
        foundlings.append(found)
        if found != last_found:
            batch_number += 1
        last_found = found
        batch.append(batch_number)
        fnames.append(fname)
    
    
    files_status = pd.DataFrame(dict(files=fnames, found=foundlings, batch=batch), index=dates)
    fetch_missing(files_status)

    df = pd.concat(map(lambda file: pd.read_csv(file, index_col='time', parse_dates=True),
                         fnames)).drop_duplicates()
    gaps = get_gaps(df, granularity)

    if len(gaps) > 0:
        print('found {} data gaps'.format(len(gaps)))
        # fetch the data for each date
        for start_date in gaps.groupby(pd.Grouper(freq='1d')).first().index:
            print('\tfetching {}'.format(start_date))
            int_ = pd.interval_range(start=start_date, periods=1, freq='1d')
            int_ = pd.Interval(int_.left[0], int_.right[0])
            int_df = load_date(ticker, granularity, int_)
            fname = audiolizer_temp_dir + '/{}-{}.csv.gz'.format(
                ticker, int_.left.strftime('%Y-%m-%d'))
            int_df.to_csv(fname, compression='gzip')

    df = pd.concat(map(lambda file: pd.read_csv(file,index_col='time', parse_dates=True, compression='gzip'),
                         fnames)).drop_duplicates()

    return df

In [ ]:
hist = get_history('BTC-USD', '2020-04-25', '2020-05-13')

In [ ]:
hist

In [ ]:
for int_ in pd.interval_range(pd.to_datetime('2020-04-26'), pd.to_datetime( '2020-05-13')):
    pass

In [ ]:
int_.left

In [ ]:
int_.right